<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/Database_07_Views.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Views and Common Table Expressions in SQLite
### Database and SQL Through Pop Culture | Brendan Shea, PhD
In the world of database management, presenting and manipulating data in a way that is both efficient and secure is of utmost importance. SQLite, a lightweight and versatile relational database management system, offers two powerful tools to achieve these goals: Views and Common Table Expressions (CTEs).

Views are virtual tables that allow you to encapsulate complex queries and present them as a single, reusable entity. They provide a way to simplify data retrieval, aggregate information, and control access to sensitive data. By creating views, you can hide the complexity of the underlying tables and provide a more user-friendly interface for querying data.

Common Table Expressions (CTEs), on the other hand, are temporary named result sets that exist within the scope of a single SQL statement. They offer a way to break down complex queries into smaller, more manageable parts. CTEs can be thought of as temporary views that are defined within a specific query and can be referenced multiple times within that query.

In this chapter, we will explore the concepts of Views and CTEs in SQLite using a real-world dataset related to music. We will be working with data from Rollingstone and Spotify, focusing on albums that appeared in any version of Rollingstone's greatest albums of all time list. (This is a greatly expanded version of the data we introduced in the previous chapter). Through practical examples and step-by-step explanations, you will learn how to create and utilize Views and CTEs to effectively present, aggregate, and secure your data.

Whether you are a beginner or an experienced SQLite user, understanding Views and CTEs is crucial for building efficient and maintainable database solutions. By the end of this chapter, you will have a solid grasp of these concepts and be able to apply them in your own projects. Let's dive in and unlock the power of Views and CTEs in SQLite!

In [1]:
!wget https://github.com/brendanpshea/database_sql/raw/main/data/rs_greatest_albums.db -q -nc
%load_ext sql
%sql sqlite:///rs_greatest_albums.db

## Data Dictionary for "Rolling Stone's Greatest Albums DB."
Before diving into views, it is good to get familiar with our data. Here's the **data dictionary** for our database. (Brendan's Note: I created this database manually, using Rollingstone's lists as a starting point, and then getting the other data from Spotify **Application Programming Interface (API)**.

#### **Table: `artists`**

This table contains information about the artists included in the RollingStone's greatest albums lists.

| **Column Name** | **Data Type** | **Description** |
| --- | --- | --- |
| `artist_id` | STRING | Unique identifier for each artist. |
| `Artist` | TEXT | Name of the artist. |
| `followers` | INTEGER | Number of followers the artist has on Spotify. |
| `genres` | TEXT | Comma-separated list of genres associated with the artist. |
| `popularity` | INTEGER | Popularity score of the artist on Spotify, ranging from 0 to 100. |

#### **Table: `albums`**

This table contains information about the albums listed in RollingStone's greatest albums lists of various years.

| **Column Name** | **Data Type** | **Description** |
| --- | --- | --- |
| `album_id` | STRING | Unique identifier for each album. |
| `Album` | TEXT | Title of the album. |
| `artist_id` | STRING | Unique identifier for the artist who created the album. |
| `Year` | INTEGER | Year the album was released. |
| `rank_2003` | INTEGER | Ranking of the album in RollingStone's 2003 list. |
| `rank_2012` | INTEGER | Ranking of the album in RollingStone's 2012 list. |
| `rank_2020` | INTEGER | Ranking of the album in RollingStone's 2020 list. |
| `rank_2023` | INTEGER | Ranking of the album in RollingStone's 2023 list. |
| **Foreign Key** |  | **FOREIGN KEY (`artist_id`) REFERENCES `artists`(`artist_id`)** |

#### **Table: `tracks`**

This table contains information about the tracks within the albums listed in the database.

| **Column Name** | **Data Type** | **Description** |
| --- | --- | --- |
| `track_id` | STRING | Unique identifier for each track. |
| `track_name` | TEXT | Name of the track. |
| `album_id` | STRING | Unique identifier for the album that the track belongs to. |
| `track_number` | INTEGER | The position of the track within the album. |
| `duration_ms` | INTEGER | Duration of the track in milliseconds. |
| **Foreign Key** |  | **FOREIGN KEY (`album_id`) REFERENCES `albums`(`album_id`)** |

### Examining the "Head" of the Data
Before trying to design our own queries, let's take a look at the **head** of the data (that is, the first few rows). We can do this using LIMIT.

In [2]:
%%sql
SELECT * FROM artists LIMIT 5;

 * sqlite:///rs_greatest_albums.db
Done.


artist_id,Artist,followers,genres,popularity
3koiLjNrgRTNbOwViDipeA,"Gaye, Marvin",5313182,"['classic soul', 'motown', 'neo soul', 'northern soul', 'quiet storm', 'soul']",71
3oDbviiivRWhXwIE8hxkVV,Beach Boys,4499081,"['baroque pop', 'classic rock', 'folk rock', 'mellow gold', 'psychedelic rock', 'rock', 'singer-songwriter', 'sunshine pop']",71
5hW4L92KnC6dX9t7tYM4Ve,"Mitchell, Joni",1461857,"['canadian singer-songwriter', 'folk', 'folk rock', 'singer-songwriter']",59
7guDJrEfX3qb6FEbdPA5qi,"Wonder, Stevie",6427524,"['motown', 'soul']",73
3WrFJ7ztbogyGnTHbHJFl2,Beatles,27469908,"['british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']",82


In [3]:
%%sql
SELECT * FROM albums LIMIT 5;

 * sqlite:///rs_greatest_albums.db
Done.


album_id,Album,artist_id,Year,rank_2003,rank_2012,rank_2020,rank_2023
7qD9oSiS3xFJsw2oSV2akp,What's Going On,3koiLjNrgRTNbOwViDipeA,1971,6,6,1,1
2CNEkSE8TADXRT2AzcEt1b,Pet Sounds,3oDbviiivRWhXwIE8hxkVV,1966,2,2,2,2
1vz94WpXDVYIEGja8cjFNa,Blue,5hW4L92KnC6dX9t7tYM4Ve,1971,30,30,3,3
6YUCc2RiXcEKS9ibuZxjt0,Songs in the Key of Life,7guDJrEfX3qb6FEbdPA5qi,1976,57,57,4,4
0ETFjACtuP2ADo6LFhL6HN,Abbey Road,3WrFJ7ztbogyGnTHbHJFl2,1969,14,14,5,5


In [4]:
%%sql
SELECT * FROM tracks LIMIT 5;

 * sqlite:///rs_greatest_albums.db
Done.


track_id,track_name,album_id,track_number,duration_ms
4cgLt753cDfsBAbpOcGnZi,What's Going On - Detroit Mix,7qD9oSiS3xFJsw2oSV2akp,1,247840
0lz97gXzH4DD822L62kLeK,What's Happening Brother - Detroit Mix,7qD9oSiS3xFJsw2oSV2akp,2,163146
0oQV9CKXruLbVUTn76eBnr,Flyin' High (In The Friendly Sky) - Detroit Mix,7qD9oSiS3xFJsw2oSV2akp,3,228693
3KAbXV1OSwthoke6BqR0y5,Save The Children - Detroit Mix,7qD9oSiS3xFJsw2oSV2akp,4,241626
1J3WjiLbPAdW0mSBTxoHQF,God Is Love - Detroit Mix,7qD9oSiS3xFJsw2oSV2akp,5,106733


You might notice a few things from this:

-   The primary keys (`artist_id`, `album_id`, `track_id`) are strings rather than the more commonly used integers. Spotify likely uses string identifiers because they are **globally unique identifiers (GUIDs)** and can be generated independently across different systems or databases without the need for coordination or central allocation. This allows for better scalability and distributed data management. These sorts of keys are common in **distributed databases** that live on multiple computers, and have large numbers of concurrent users.
-   The `genres` column in the `artists` table contains a list of genres associated with each artist. This indicates that an artist can belong to multiple genres, providing a more comprehensive categorization. The genres are stored as a comma-separated list within square brackets, resembling a JSON array format. This structure allows for efficient storage and parsing of multiple genres per artist. However, we'll need to be careful when writing SQL queries to access this.
-  The `albums` table includes ranking columns (`rank_2003`, `rank_2012`, `rank_2020`, `rank_2023`) representing the album's position in Rollingstone's greatest albums lists for different years. The presence of multiple ranking columns suggests that the rankings have evolved over time, and the dataset captures these changes.
- The `duration_ms` column in the `tracks` table stores the duration of each track in milliseconds. This level of granularity allows for precise analysis and calculations related to track lengths. However, it isn't very friendly to human readers. We'll create a view to fix this.

## What is a VIEW?

In SQL, a **view** is a virtual table that is based on the result of a SELECT statement. It acts as a stored query that can be accessed and manipulated as if it were a regular table. Views provide a way to simplify complex queries, present data in a more readable format, and enhance data security by limiting access to specific columns or rows.

The general syntax for creating a view in SQ is as follows:

```sql
CREATE [TEMP | TEMPORARY] VIEW [IF NOT EXISTS] view_name AS
SELECT column1, column2, ...
FROM table_name
WHERE condition;
```

Let's break down the components of the CREATE VIEW statement:

-   `TEMP` or `TEMPORARY`: These keywords are optional and are used to create a temporary view that exists only for the duration of the current session.
-   `IF NOT EXISTS`: This clause is optional and prevents an error from occurring if a view with the same name already exists.
-   `view_name`: This is the name you assign to the view. It follows the same naming conventions as tables.
-   `AS`: This keyword precedes the SELECT statement that defines the view.
-   `SELECT`: This is the query that retrieves the data for the view. It can include columns from one or more tables, along with any desired conditions, joins, or aggregations.

Once a view is created, you can query it just like you would query a regular table. The view's result set is dynamically generated each time it is accessed, based on the underlying SELECT statement.

Here's a simple example of creating a view:

In [5]:
%%sql
DROP VIEW IF EXISTS artist_info;

CREATE VIEW album_info AS
SELECT
  Albums.album,
  Albums.year,
  Artist.artist
FROM Albums
  JOIN Artists ON Albums.artist_id = Artists.artist_id;

 * sqlite:///rs_greatest_albums.db
Done.
Done.


[]

In this example, we create a view named `album_info` that combines information from the `albums` and `artists` tables. The view includes the `Album` and `Year` columns from the `albums` table and the `Artist` column from the `artists` table. The tables are joined based on the `artist_id` foreign key relationship.

Now, whenever we query the `album_info` view, it will return the result set as defined by the SELECT statement, providing a convenient way to access the combined album and artist information.

In [6]:
%%sql
SELECT * FROM album_info LIMIT 5;

 * sqlite:///rs_greatest_albums.db
Done.


Album,Year,Artist
What's Going On,1971,"Gaye, Marvin"
Pet Sounds,1966,Beach Boys
Blue,1971,"Mitchell, Joni"
Songs in the Key of Life,1976,"Wonder, Stevie"
Abbey Road,1969,Beatles


Views offer several benefits, such as:

-   Simplifying complex queries by encapsulating them into a reusable entity.
-   Providing a layer of abstraction and hiding the complexity of the underlying tables.
-   Enhancing data security by restricting access to specific columns or rows.
-   Enabling consistent and centralized data access across multiple queries or applications.

In the following sections, we will explore various practical examples of using views in SQLite to present, aggregate, and secure data from the music dataset.

## Views for Presenting Data

Views provide a powerful way to present data in a more user-friendly and meaningful manner. By creating views, you can customize the appearance of the data, combine information from multiple tables, and present it in a way that is easier to understand and interpret. Let's look at a few examples using the music dataset.

### Greatest albums, 2023 version only

Suppose you want to create a view that displays only the albums that appeared in the 2023 version of Rollingstone's greatest albums list. Here's how you can achieve that:

In [7]:
%%sql
DROP VIEW IF EXISTS greatest_albums_2023;

CREATE VIEW greatest_albums_2023 AS
SELECT
  Albums.album,
  Artists.artist,
  Albums.year,
  Albums.rank_2023
FROM Albums
JOIN Artists ON Albums.artist_id = artists.artist_id
WHERE rank_2023 IS NOT NULL
ORDER BY rank_2023;

 * sqlite:///rs_greatest_albums.db
Done.
Done.


[]

In this example, the `greatest_albums_2023` view retrieves the `Album`, `Artist`, `Year`, and `rank_2023` columns from the `albums` and `artists` tables. The tables are joined based on the `artist_id` foreign key relationship. The `WHERE` clause filters the results to include only the albums that have a non-null value in the `rank_2023` column. Finally, the `ORDER BY` clause sorts the albums based on their 2023 ranking.

Now, you can query the `greatest_albums_2023` view to retrieve the list of albums from the 2023 version of Rollingstone's list:

In [8]:
%%sql
SELECT * FROM greatest_albums_2023 LIMIT 10;

 * sqlite:///rs_greatest_albums.db
Done.


Album,Artist,Year,rank_2023
What's Going On,"Gaye, Marvin",1971,1
Pet Sounds,Beach Boys,1966,2
Blue,"Mitchell, Joni",1971,3
Songs in the Key of Life,"Wonder, Stevie",1976,4
Abbey Road,Beatles,1969,5
Nevermind,Nirvana,1991,6
Rumours,Fleetwood Mac,1977,7
Purple Rain,Prince and the Revolution,1984,8
Blood on the Tracks,"Dylan, Bob",1975,9
The Miseducation of Lauryn Hill,"Hill, Lauryn",1998,10


### Presenting track information with reading Minutes and Seconds

Let's say you want to present the track information in a more readable format by converting the duration from milliseconds to minutes and seconds. Here's how you can create a view for that:

In [12]:
%%sql
DROP VIEW IF EXISTS track_info;

CREATE VIEW track_info AS
SELECT
  Tracks.track_name,
  Albums.album,
  Tracks.track_number,
       PRINTF('%d:%02d', duration_ms / 1000 / 60, duration_ms / 1000 % 60) AS duration
FROM Tracks
  JOIN Albums ON Tracks.album_id = Albums.album_id;

 * sqlite:///rs_greatest_albums.db
Done.
Done.


[]

In this view, the `track_info` retrieves the `track_name`, `album_id`, `track_number`, and `duration` columns from the `tracks` table. The `printf` function is used to format the `duration_ms` column into a string representation of minutes and seconds (MM:SS format). The `duration_ms` value is divided by 1000 to convert it from milliseconds to seconds, and then the minutes and seconds are extracted using integer division and modulo operations.

You can now query the `track_info` view to retrieve the track information with the formatted duration:

In [13]:
%%sql
SELECT * FROM track_info LIMIT 5;

 * sqlite:///rs_greatest_albums.db
Done.


track_name,Album,track_number,duration
What's Going On - Detroit Mix,What's Going On,1,4:07
What's Happening Brother - Detroit Mix,What's Going On,2,2:43
Flyin' High (In The Friendly Sky) - Detroit Mix,What's Going On,3,3:48
Save The Children - Detroit Mix,What's Going On,4,4:01
God Is Love - Detroit Mix,What's Going On,5,1:46


### Albums with biggest change from 2003 to 2023

To identify the albums that had the biggest change in ranking from 2003 to 2023, you can create a view that calculates the difference between the rankings:

In [19]:
%%sql
DROP VIEW IF EXISTS biggest_rank_change;

CREATE VIEW biggest_rank_change AS
SELECT
  Albums.album,
  Artists.artist,
  Albums.year,
  Albums.rank_2003,
  Albums.rank_2023,
       rank_2003 - rank_2023 AS rank_change
FROM
  Albums
  JOIN Artists ON Albums.artist_id = Artists.artist_id
WHERE
  rank_2003 IS NOT NULL
  AND rank_2023 IS NOT NULL
ORDER BY ABS(rank_change) DESC
LIMIT 10;

 * sqlite:///rs_greatest_albums.db
Done.
Done.


[]

In this view, the `biggest_rank_change` retrieves the `Album`, `Artist`, `rank_2003`, `rank_2023`, and `rank_change` columns. The `rank_change` column is calculated by subtracting the `rank_2023` from the `rank_2003`. The `WHERE` clause filters the results to include only albums that have non-null values for both `rank_2003` and `rank_2023`. The `ORDER BY` clause sorts the albums based on the absolute value of the `rank_change` in descending order. Finally, the `LIMIT` clause retrieves only the top 10 albums with the biggest rank change.

You can query the `biggest_rank_change` view to see the albums with the most significant changes in ranking between 2003 and 2023:

In [20]:
%%sql
SELECT * FROM biggest_rank_change

 * sqlite:///rs_greatest_albums.db
Done.


Album,Artist,Year,rank_2003,rank_2023,rank_change
Voodoo,D'Angelo,2000,480,28,452
Aquemini,OutKast,1998,500,49,451
The Blueprint,Jay-Z,2001,456,50,406
Kid A,Radiohead,2000,420,20,400
Enter the Wu_Tang: 36 Chambers,Wu Tang Clan,1993,382,27,355
Illmatic,Nas,1994,396,44,352
Live Through This,Hole,1994,458,106,352
Maggot Brain,Funkadelic,1971,478,136,342
The Score,Fugees,1996,469,134,335
Heart Like a Wheel,"Ronstadt, Linda",1975,163,490,-327


This view has some limitations (for example, it doesn't account for albums that fell out of the rankings altogether). However, it gives a good sense of how critics views have changed over time. For example, many of the albums the had the biggest increases in rank (by D'Angelo, Jay-Z, Radiohead, and Outkast) were still relatively "new" in 2003. However, looking back, it appears they have "stood the test of time."